In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/My Drive/IndependentResearch/Trading_Bot/tData/"
output_dir_path = "/content/drive/My Drive/IndependentResearch/Trading_Bot/Results/"

In [ ]:

import pandas
!pip install yfinance
!pip install plotly
!pip install alpaca-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 708.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.13
    Uninstalling pydantic-1.10.13:
      Successfully uninstalled pydantic-1.10.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python

In [ ]:
# import functions
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import time
import yfinance as yf

In [ ]:
#Grab Data and put into data frame
def grab_data(ticker,period,interval):
  ticker = ticker
  data = yf.download(tickers=ticker, period=period, interval=interval)
  num_rows = data.shape[0]
  data.to_csv(data_path + ticker + "_" + period + "_" + interval + "_" + "original_input.csv")

  return data


In [ ]:
def SMA_Crossover(data, ticker, short_window,long_window,pos_SMA,neg_SMA):
  # adds short SMA parameter to input data

    short_SMA = data['Close'].rolling(window = short_window).mean()
    data["Short SMA"] = short_SMA


  # adds long SMA parameter to input data

    long_SMA = data['Close'].rolling(window=long_window).mean()
    data["Long SMA"] = long_SMA

    # calculates SMA percent
    data['SMA Calc'] = ""
    num_rows = data.shape[0]
    for i in range(num_rows):
      data['SMA Calc'].iloc[i] = 1 - data['Short SMA'].iloc[i]/data['Long SMA'].iloc[i]

    # Calculate a buy sell or a hold
    buying_period = False
    data['BSH'] = ""
    for i in range(num_rows):
      # calculate_SMA = 1 - data['Short SMA'].iloc[i]/data['Long SMA'].iloc[i]
      calculate_SMA = data['SMA Calc'].iloc[i]
      current_price = data['Close'].iloc[i]

      if calculate_SMA >= pos_SMA and buying_period == False:
        data['BSH'].iloc[i] = "BUY"
        buy_price = data['Close'].iloc[i]
        buying_period = True

      elif calculate_SMA <= neg_SMA and buying_period == True:
        data['BSH'].iloc[i] = "SELL"
        buying_period = False

      else:
        data['BSH'].iloc[i] = "HOLD"

    data.to_csv(data_path + ticker + "after_SMA_strategy.csv")
    return data



In [ ]:
# create data of all the buy sells the bot does
def resulting_data(data,ticker):
  # create resulting data
  num_rows = data.shape[0]
  visual_data = data.reset_index()
  visual_data = visual_data.rename(columns={visual_data.columns[0]: 'Datetime'})
  results = pd.DataFrame(columns=['Date of Buy','Buy Price', 'Date of Sell', 'Sell Price'])
  j = 0

  # create result data
  for i in range(num_rows):
    if data['BSH'].iloc[i] == "BUY":
      results = results.append({'Buy Price' : data['Close'].iloc[i] }, ignore_index = True)
      results['Date of Buy'].iloc[j] = visual_data['Datetime'].iloc[i]

    elif data['BSH'].iloc[i] == "SELL":
      results['Sell Price'].iloc[j] = data['Close'].iloc[i]
      results['Date of Sell'].iloc[j] = visual_data['Datetime'].iloc[i]
      j = j + 1
  print(results)
  # calculate difference of buy and sell
  results['Difference'] = ' '

  for i in range(results.shape[0]):
    results["Difference"].iloc[i] = results['Sell Price'].iloc[i] - results['Buy Price'].iloc[i]

  # drop bottom row if the bot buys but doesnt have enough data to sell
  results.dropna(subset=['Sell Price'], inplace=True)
  results.dropna(subset=['Date of Sell'], inplace=True)

  # create csv and save it
  results.to_csv(output_dir_path + ticker + '_results.csv')

  # return results of backtesting
  bots_sum = 0
  if results.empty:
    raise Exception("The bot did not place any buy orders")
  else:
    for i in range(results.shape[0]):
      bots_sum = results['Difference'].iloc[i] + bots_sum
      bots_expected_returns = bots_sum / results['Buy Price'].iloc[0] * 100

    reg_sum = results['Sell Price'].iloc[-1] - results['Buy Price'].iloc[0]
    reg_returns = reg_sum / results['Buy Price'].iloc[0] * 100

  return results, bots_sum, bots_expected_returns, reg_sum, reg_returns

In [ ]:
# for live loop
# https://www.geeksforgeeks.org/how-to-update-a-plot-on-same-figure-during-the-loop/

# visualize the data with a chart
def visualize(results, data, ticker):
  # drop index to be able to drab the date
  visual_data = data.reset_index()
  visual_data = visual_data.rename(columns={visual_data.columns[0]: 'Datetime'})


  # turn to datetime attribute
  results['Date of Buy'] = pd.to_datetime(results['Date of Buy'])
  results['Date of Sell'] = pd.to_datetime(results['Date of Sell'])

  # set variables
  market_data = visual_data['Close']
  time_series = visual_data['Datetime']

  fig = plt.figure(figsize= (10,5))

  # plot the stocks data and the points at which the bot buys and sells
  plt.plot(time_series, market_data, color = 'black')
  plt.plot(results['Date of Buy'], results['Buy Price'], 'o', color='green')
  plt.plot(results['Date of Sell'], results['Sell Price'],'o', color='red')

  # create variables
  buy_point = []
  sell_point = []
  x_val = []
  y_val = []

  # plot lines in between the buy and sell, make lines green or red depending on whether its a positive or negative slope
  for i in range(results.shape[0]):
    buy_point = [results['Date of Buy'].iloc[i], results['Buy Price'].iloc[i]]
    sell_point = [results['Date of Sell'].iloc[i], results['Sell Price'].iloc[i]]
    x_val = [buy_point[0], sell_point[0]]
    y_val = [buy_point[1], sell_point[1]]
    # plot lines
    if results['Sell Price'].iloc[i] - results['Buy Price'].iloc[i] >= 0:
      plt.plot(x_val, y_val, linestyle="-" , color = 'green')
    else:
      plt.plot(x_val, y_val, linestyle="-" , color = 'red')

  # plot chart
  plt.xlabel("Time")
  plt.ylabel("Price")
  plt.title(ticker + ' Market Data')
  plt.grid(True)
  plt.show()

  fig.savefig(output_dir_path + 'visualize_result_of' + ticker + '.png')

In [ ]:
ticker = 'AAPL'
period = '1y'
interval = '1h'


data = grab_data(ticker, period, interval)
trading_strategy_data = SMA_Crossover(data,'AAPL', 10,60,.01,-.05)
results = resulting_data(trading_strategy_data,'AAPL')
result_df = results[0]
print("Your backtesting result is a return of " , results[1])
print("The percent change from the original buy value is " , results[2] , '%')

print("Your buy and hold result is a return of" , results[3])
print("The percent change from the original buy value is " , results[4] , '%')

visualize(result_df,data,ticker)



[*********************100%%**********************]  1 of 1 completed


<ipython-input-7-5755a7791c33>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['SMA Calc'].iloc[i] = 1 - data['Short SMA'].iloc[i]/data['Long SMA'].iloc[i]
<ipython-input-7-5755a7791c33>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BSH'].iloc[i] = "HOLD"
<ipython-input-7-5755a7791c33>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BSH'].iloc[i] = "BUY"
<ipython-input-7-5755a7791c33>:33: SettingWithCopyWarning

AttributeError: ignored